In [ ]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_predict
import numpy as np
from scikeras.wrappers import KerasClassifier
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
# Reorganizing images into new directories for multiple classification


os.makedirs('glioma_tumor', exist_ok=True)
os.makedirs('meningioma_tumor', exist_ok=True)
os.makedirs('pituitary_tumor', exist_ok=True)
os.makedirs('normal', exist_ok=True)

for dataset in ['train', 'val', 'test']:
    for category in ['tumor', 'nontumor']:
        category_path = os.path.join(dataset, category)
        if os.path.exists(category_path):
            for img in os.listdir(category_path):
                if img.startswith('G'):
                    shutil.move(os.path.join(category_path, img), 'glioma_tumor')
                elif img.startswith('M'):
                    shutil.move(os.path.join(category_path, img), 'meningioma_tumor')
                elif img.startswith('P'):
                    shutil.move(os.path.join(category_path, img), 'pituitary_tumor')
                elif img.startswith('N'):
                    shutil.move(os.path.join(category_path, img), 'normal')

shutil.rmtree('train')
shutil.rmtree('val')
shutil.rmtree('test')

for dir_name in ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'normal']:
    count = len(os.listdir(dir_name))
    print(f"{dir_name} has {count} images")

In [ ]:
# Performing a train, val, test split 

def split_data(class_name, split_ratios=(0.8, 0.1, 0.1)):
    files = os.listdir(class_name)
    train_files, temp_files = train_test_split(files, train_size=split_ratios[0], random_state=42)
    val_files, test_files = train_test_split(temp_files, train_size=split_ratios[1]/(split_ratios[1] + split_ratios[2]), random_state=42)

    os.makedirs(f'train/{class_name}', exist_ok=True)
    os.makedirs(f'val/{class_name}', exist_ok=True)
    os.makedirs(f'test/{class_name}', exist_ok=True)

    for file in train_files:
        shutil.move(os.path.join(class_name, file), os.path.join(f'train/{class_name}', file))
    for file in val_files:
        shutil.move(os.path.join(class_name, file), os.path.join(f'val/{class_name}', file))
    for file in test_files:
        shutil.move(os.path.join(class_name, file), os.path.join(f'test/{class_name}', file))

    return len(train_files), len(val_files), len(test_files)

for class_name in ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'normal']:
    train_count, val_count, test_count = split_data(class_name)
    print(f"{class_name} - Train: {train_count}, Val: {val_count}, Test: {test_count}")


In [ ]:
# Setting up image generators for multiclass classification with augmentation for training and normalization for validation & testing

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    'val',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    'test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)
